# 문장 단위 Word2Vec

- Ubuntu 18.04 LTS (Mecab 때문에 윈도우에서는 안돌아갑니다!)
- 52GB RAM, 8 Cores
- 문장 토크나이징에는 꼬꼬마, 명사 추출에는 Mecab 사용

In [1]:
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
import matplotlib.pyplot as plt

from konlpy.tag import Mecab, Kkma
from gensim.models import Word2Vec

## 1.데이터 전처리

In [ ]:
# 중복 제거
data = pd.read_csv("./data/data.csv").dropna().drop_duplicates("Content") # 중복 제거

# 카테고리 정렬, 불필요한 컬럼 드랍, 한글 및 띄어쓰기만 남겨주기
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자 삭제
data = data.assign(
    Content = data.Content.apply(lambda x: x.strip()[1:-1]).apply(lambda x: hangul.sub("", x).strip()),
    Category = data.Category.apply(lambda x: x.strip()).replace({
        "외교/통일/국토":"외교/통일/국방", "안전/환제":"안전/환경", "보건복업지":"보건복지"
    })).drop(columns = ["Unnamed: 0", "Unnamed: 0.1"]).dropna() # 간단한 전처리

# 문장 단위로 훈련할 것이므로 유의미하게 분석 가능한 수준의 문서만 남김
data = data[(data.Content.apply(len) >= 30) & (data.Content.apply(lambda x: len(x.split(" ")) >= 5))] # 30글자 5어절 이상만

# 데이터 저장
data.to_csv("data/data.csv", index=False, encoding = "utf-8")

In [2]:
data = pd.read_csv("data/data.csv", encoding= 'utf-8')

## 2. 모델 훈련

In [2]:
class myKkma(Kkma):
     def sentences(self, phrase):
        """
        Sentence detection: 예외처리 추가
        """
        try:
            sentences = self.jki.morphAnalyzer(phrase)
            if not sentences: return []
            return [sentences.get(i).getSentence() for i in range(sentences.size())]
        except:
            return []

kkma, mecab = myKkma(), Mecab()

In [3]:
def modelTrain(cat):
    df = data[data.Category == cat] # 카테고리 필터
    result = df.Content.apply(lambda x: "|".join(kkma.sentences(x))) # 문장 단위로 구분
    result.reset_index().drop("index", axis=1).to_csv(f"data/{filename}.csv", index=False) # 문장 파싱 결과 저장
    
    tmp = pd.read_csv(f"data/{filename}.csv").dropna()
    nouns = [mecab.nouns(sentence) for sentence in tmp.Content.apply(lambda x: x.split("|")).sum()] # 명사 추출
    model = Word2Vec(nouns, window=3, min_count=3, size=100, sg=1) # 모델 훈련
    model.save(f"models/skipgram/{filename}.model") # 모델 저장

In [5]:
def loadAndVal(path, topn=50):
    mostSimilar = Word2Vec.load(path).wv.most_similar("정의", topn=topn)
    
    result = pd.DataFrame({
        "category":path.split("/")[-1].split(".")[0],
        "rank":np.arange(1,51),
        "keyword":[word for word, distance in mostSimilar],
        "distance":[distance for word, distance in mostSimilar],
        
        
    })
    
    return result

In [ ]:
# 실제로는 7개 세션에서 나눠서 돌렸습니다
for cat in data.Category.unique():
    modelTrain(cat)

In [6]:
sgModels = [f"./models/skipgram/{filename}" for filename in os.listdir("./models/skipgram/")]
sgResult = pd.concat(list(pd.Series(sgModels).apply(loadAndVal)))

In [10]:
sgResult.pivot(index = "rank", columns = "category", values = "keyword").to_csv("")

category,경제민주화,교통건축국토,기타,농산어촌,문화예술체육언론,미래,반려동물,보건복지,성장동력,안전환경,외교통일국방,육아교육,인권성평등,일자리,저출산고령화대책,정치개혁,행정
rank,,,,,,,,,,,,,,,,,
1,구현,구현,구현,촛불,불의,여긴,바탕,만인,상식,구현,독선,만인,구현,평등,평화,공의,구현
2,구호,만인,법치,인권,삽,권선징악,조장,입각,존중,공의,강자,출발점,공의,공정,자국민,구현,공의
3,만인,공평,갈망,주권,구현,조자,세분,공화국,이성,민중,건전,구현,법질서,구현,공정,공정,공정
4,공서,반칙,공의,민주,사제단,구현,제시,공정,기득,기강,우연,수호,첩경,정당화,애국,준법정신,반칙
5,공의,공의,법질서,평등,강자,법치주의,비상식,개법,대립,사법부,공정,주창,솔로몬,함의,민주,여신상,좌지우지
6,승승,박수,법치주의,탄생,민중,감수성,세계인,공평,관행,입법부,공정성,표방,여신상,주권자,부강,사제단,배신
7,반칙,추상,여신상,응원,평등,공의,과학,다복,추구,법치,법치주의,평등,실현,다수결,잡종,법치주의,근간
8,불의,취임사,신의칙,권력,타협,결백,대중,법치주의,이념,법치주의,물음,승복,법치국,공평,후손,수호자,법치주의
9,실천,특권,불의,역사,백계,공평,빌미,공산주의,강자,만인,참패,대한,순리,취임사,서양,정의로운,여신상


In [12]:
sgResult.to_csv("정의단어/정의단어_문장단위.csv", index=False)
sgResult.pivot(index = "rank", columns = "category", values = "keyword").to_csv("정의단어/피봇테이블_문장단위.csv")

## 3. 결과 분석